Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer




In [2]:
#uploading training dataset
test_data= pd.read_csv("UNSW_NB15_testing-set.csv",header=None)
print('-------Test data--------')
print(test_data[43].value_counts())
print(test_data.shape)
print('-------------------------')
test_data.drop(test_data.index[0],inplace = True)

-------Test data--------
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: 43, dtype: int64
(175341, 45)
-------------------------


In [3]:
test_data.drop(test_data[test_data[43]=='Generic'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Fuzzers'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Reconnaissance'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Analysis'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Backdoor'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Shellcode'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Worms'].index, inplace = True)
print(test_data[43].value_counts())

Normal      55999
Exploits    33393
DoS         12264
Name: 43, dtype: int64


In [4]:
#uploading training dataset
train_data= pd.read_csv("UNSW_NB15_training-set.csv",header= None)
print('-------Train data--------')
print(train_data[43].value_counts())
print(len(train_data))
print('-------------------------')
train_data.drop(train_data.index[0],inplace = True)
train_data

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


-------Train data--------
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
attack_cat            1
Name: 43, dtype: int64
82333
-------------------------


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
1,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
2,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
3,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
4,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
5,5,0.00001,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,82328,5e-06,udp,-,INT,2,0,104,0,200000,...,1,2,0,0,0,2,1,0,Normal,0
82329,82329,1.1061,tcp,-,FIN,20,8,18062,354,24.4101,...,1,1,0,0,0,3,2,0,Normal,0
82330,82330,0,arp,-,INT,1,0,46,0,0,...,1,1,0,0,0,1,1,1,Normal,0
82331,82331,0,arp,-,INT,1,0,46,0,0,...,1,1,0,0,0,1,1,1,Normal,0


In [5]:
train_data.drop(train_data[train_data[43]=='Generic'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Fuzzers'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Reconnaissance'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Backdoor'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Analysis'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Shellcode'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Worms'].index, inplace = True)
print(train_data[43].value_counts())

Normal      37000
Exploits    11132
DoS          4089
Name: 43, dtype: int64


In [6]:
#preprocessing
#Evaluation of the training dataset
train_data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
dtype: int64

In [7]:

train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 45 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       52221 non-null  object
 1   1       52221 non-null  object
 2   2       52221 non-null  object
 3   3       52221 non-null  object
 4   4       52221 non-null  object
 5   5       52221 non-null  object
 6   6       52221 non-null  object
 7   7       52221 non-null  object
 8   8       52221 non-null  object
 9   9       52221 non-null  object
 10  10      52221 non-null  object
 11  11      52221 non-null  object
 12  12      52221 non-null  object
 13  13      52221 non-null  object
 14  14      52221 non-null  object
 15  15      52221 non-null  object
 16  16      52221 non-null  object
 17  17      52221 non-null  object
 18  18      52221 non-null  object
 19  19      52221 non-null  object
 20  20      52221 non-null  object
 21  21      52221 non-null  object
 22  22      52221 non-null

In [8]:
train_data=train_data.drop([2,3,43],axis=1)
train_data=train_data.drop([0],axis=1)
test_data=test_data.drop([2,3,43],axis=1)
test_data=test_data.drop([0],axis=1)

In [9]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
'''train_data[43] = le.fit_transform(train_data[43])
test_data[43] = le.fit_transform(test_data[43])'''
train_data[4] = le.fit_transform(train_data[4])
test_data[4] = le.fit_transform(test_data[4])

In [10]:
train_data = train_data.apply(pd.to_numeric)
test_data = test_data.apply(pd.to_numeric)

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       52221 non-null  float64
 1   4       52221 non-null  int32  
 2   5       52221 non-null  int64  
 3   6       52221 non-null  int64  
 4   7       52221 non-null  int64  
 5   8       52221 non-null  int64  
 6   9       52221 non-null  float64
 7   10      52221 non-null  int64  
 8   11      52221 non-null  int64  
 9   12      52221 non-null  float64
 10  13      52221 non-null  float64
 11  14      52221 non-null  int64  
 12  15      52221 non-null  int64  
 13  16      52221 non-null  float64
 14  17      52221 non-null  float64
 15  18      52221 non-null  float64
 16  19      52221 non-null  float64
 17  20      52221 non-null  int64  
 18  21      52221 non-null  int64  
 19  22      52221 non-null  int64  
 20  23      52221 non-null  int64  
 21  24      52221 non-null  float64
 22

In [12]:
'''list=[1,9,12,13,16,17,18,19,24,25,26]
for x in list:
  '''
train_data.round(decimals=2)
train_data = train_data.astype(int)
test_data.round(decimals=2)
test_data = test_data.astype(int)
train_data

,1,4,5,6,7,8,9,10,11,12,...,34,35,36,37,38,39,40,41,42,44
1,0,3,2,0,496,0,90909,254,0,180363632,...,1,1,2,0,0,0,1,2,0,0
2,0,3,2,0,1762,0,125000,254,0,881000000,...,1,1,2,0,0,0,1,2,0,0
3,0,3,2,0,1068,0,200000,254,0,854400000,...,1,1,3,0,0,0,1,3,0,0
4,0,3,2,0,900,0,166666,254,0,600000000,...,2,1,3,0,0,0,2,3,0,0
5,0,3,2,0,2126,0,100000,254,0,850400000,...,2,1,3,0,0,0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,0,3,2,0,104,0,200000,254,0,83200000,...,1,1,2,0,0,0,2,1,0,0
82329,1,2,20,8,18062,354,24,254,252,124104,...,1,1,1,0,0,0,3,2,0,0
82330,0,3,1,0,46,0,0,0,0,0,...,1,1,1,0,0,0,1,1,1,0
82331,0,3,1,0,46,0,0,0,0,0,...,1,1,1,0,0,0,1,1,1,0


In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1       52221 non-null  int32
 1   4       52221 non-null  int32
 2   5       52221 non-null  int32
 3   6       52221 non-null  int32
 4   7       52221 non-null  int32
 5   8       52221 non-null  int32
 6   9       52221 non-null  int32
 7   10      52221 non-null  int32
 8   11      52221 non-null  int32
 9   12      52221 non-null  int32
 10  13      52221 non-null  int32
 11  14      52221 non-null  int32
 12  15      52221 non-null  int32
 13  16      52221 non-null  int32
 14  17      52221 non-null  int32
 15  18      52221 non-null  int32
 16  19      52221 non-null  int32
 17  20      52221 non-null  int32
 18  21      52221 non-null  int32
 19  22      52221 non-null  int32
 20  23      52221 non-null  int32
 21  24      52221 non-null  int32
 22  25      52221 non-null  int32
 23  26      522

In [14]:
'''X=train_data.drop('attack_cat',axis=1)
Y=train_data['attack_cat']'''

"X=train_data.drop('attack_cat',axis=1)\nY=train_data['attack_cat']"

In [15]:
features = train_data.iloc[:,0:40]
target = train_data.iloc[:,40]
X_Train,X_Test,Y_Train,Y_Test=train_test_split(features,target,test_size = 0.2,random_state=42)

In [16]:
X = X_Train
Y = Y_Train
C = Y_Test
T = X_Test
print(Y)
print(X)

67588    0
39050    0
65875    0
4905     1
36104    0
        ..
25757    0
74844    0
68270    0
1289     1
30268    0
Name: 44, Length: 41776, dtype: int32
       1   4   5   6      7     8       9    10   11         12  ...  33  34  \
67588   1   2  14  10   5154   830      19  254  252      32327  ...   1   1   
39050   0   1   2   2    146   178    2964   31   29     577075  ...   8   1   
65875   0   2  10   8    828  1308      19   62  252       6719  ...   1   2   
4905    0   3   2   0    200     0  200000  254    0  160000000  ...   3   3   
36104   0   2  52  42  37232  3380    2586   31   29    8125500  ...   4   1   
...    ..  ..  ..  ..    ...   ...     ...  ...  ...        ...  ...  ..  ..   
25757   0   2  22  24   1920  4312    6108   31   29    1990498  ...   3   1   
74844   0   4   2   0     90     0  142857  254    0   51428572  ...   1   1   
68270   0   2  10   8    866  1144      17   62  252       6536  ...   1   1   
1289   28   3  20   0   1280     0       

In [17]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from tensorflow.keras.preprocessing import sequence
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
trainX = scaler.fit_transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

In [19]:
scaler = StandardScaler()
# transform data
testT = scaler.fit_transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

In [20]:
y_train = np.array(Y)
y_test = np.array(C)

In [21]:

X_train = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0],1, testT.shape[1]))

In [22]:


batch_size = 64

# 1. define the network
model = Sequential()
model.add(SimpleRNN(8,input_dim=40, return_sequences=True)) 
model.add(Dropout(0.1))
model.add(SimpleRNN(8, return_sequences=False)) 
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=50)

Epoch 1/50
653/653 [==============================] - 6s 3ms/step - loss: 0.4761 - accuracy: 0.7526
Epoch 2/50
653/653 [==============================] - 2s 3ms/step - loss: 0.2480 - accuracy: 0.8939
Epoch 3/50
653/653 [==============================] - 2s 3ms/step - loss: 0.2240 - accuracy: 0.9037
Epoch 4/50
653/653 [==============================] - 2s 3ms/step - loss: 0.2086 - accuracy: 0.9120
Epoch 5/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1979 - accuracy: 0.9186
Epoch 6/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1902 - accuracy: 0.9223
Epoch 7/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1882 - accuracy: 0.9222
Epoch 8/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1837 - accuracy: 0.9227
Epoch 9/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1757 - accuracy: 0.9259
Epoch 10/50
653/653 [==============================] - 2s 4ms/step - loss: 0.1675 - accuracy: 0.9321

In [24]:
loss, accuracy = model.evaluate(X_test, y_test)
y_pred = model.predict_classes(X_test)

327/327 [==============================] - 1s 2ms/step - loss: 0.1191 - accuracy: 0.9577


C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
y_train1 = y_test
y_pred = model.predict_classes(X_test)
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")
print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("racall")
print("%.3f" %recall)
print("precision")
print("%.3f" %precision)
print("f1score")
print("%.3f" %f1)
cm = metrics.confusion_matrix(y_train1, y_pred)
print(cm)
print("==============================================")

confusion matrix
----------------------------------------------
accuracy
0.958
racall
0.958
precision
0.959
f1score
0.957
[[7275   59]
 [ 383 2728]]


In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_train1, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      7347
           1       0.94      0.90      0.92      3233

    accuracy                           0.95     10580
   macro avg       0.95      0.94      0.94     10580
weighted avg       0.95      0.95      0.95     10580

